## 10.2 Segmentation data

In [0]:
import pandas as pd
seg_df = pd.read_csv('http://bit.ly/PMR-ch5')
seg_df.head()

In [0]:
from google.colab import files

f = files.upload()

In [0]:
import pandas as pd

seg_df = pd.read_csv('segment_dataframe_Python_intro_Ch5.csv',
                     index_col=0)
seg_df.head()

In [0]:
seg_df['is_female'] = seg_df.gender == 'female'
seg_sub = seg_df.drop(['Segment', 'gender'], axis=1)
seg_sub.head()

In [0]:
from sklearn import preprocessing

seg_sc = pd.DataFrame(preprocessing.scale(seg_sub),
                      columns=seg_sub.columns)
seg_sc.head()

### 10.3 Check function

In [0]:
pd.pivot_table(seg_sub, index=seg_df.Segment)

In [0]:
import numpy as np

# Output not shown
pd.pivot_table(seg_sub, index=seg_df.Segment,
               aggfunc=[np.mean, np.std]).unstack()

In [0]:
pd.pivot_table(seg_sub, index=seg_df.Segment,
               aggfunc=lambda x: np.percentile(x, 95))

In [0]:
def check_clusters(data, labels):
  return pd.pivot_table(data,
                        index=labels)

check_clusters(seg_sub, seg_df.Segment)

### 10.3.2 Hierarchical clustering and distances

In [0]:
# Vector of differences
np.array([1, 2, 3]) - np.array([2, 3, 2])

In [0]:
# Sum of the squared distances
np.sum((np.array([1, 2, 3]) - np.array([2, 3, 2]))**2)

In [0]:
# Root sum of the squared distances
np.sqrt(np.sum((np.array([1, 2, 3]) - np.array([2, 3, 2]))**2))

In [0]:
from scipy.spatial import distance

distance.pdist([np.array([1, 2, 3]), np.array([2, 3, 2])])

In [0]:
distance.pdist(seg_sc.iloc[:3])

In [0]:
distance.squareform(distance.pdist(seg_sc.iloc[:3]))

In [0]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.hist(distance.pdist(seg_sub))
plt.title('Pairwise distances from unscaled data')
plt.xlabel('Distance')
plt.ylabel('Count')
plt.subplot(1,2,2)
plt.hist(distance.pdist(seg_sc))
plt.xlabel('Distance')
plt.ylabel('Count')
plt.title('Pairwise distances from scaled data')

In [0]:
from scipy.cluster import hierarchy

linkages = hierarchy.linkage(seg_sc, method='ward')

In [0]:
hierarchy.dendrogram(linkages)
plt.show()

In [0]:
hierarchy.dendrogram(linkages, orientation='top',
                     truncate_mode='lastp', p=20)
plt.show()

In [0]:
plt.subplot(1,2,1)
hierarchy.dendrogram(linkages, leaf_rotation=0)
plt.xlim((0,200))
plt.subplot(1,2,2)
hierarchy.dendrogram(linkages, leaf_rotation=0)
plt.xlim((2800, 3000))
plt.show()

In [0]:
# Similar
seg_sub.loc[[17, 51]]

In [0]:
# Dissimilar
seg_sub.loc[[163, 88]]

In [0]:
# Dissimilar
seg_sub.loc[[17,163]]

In [0]:
hierarchy.cophenet(linkages, distance.pdist(seg_sc))[0]

In [0]:
# Not shown
hierarchy.dendrogram(linkages, color_threshold=9)
plt.show()

In [0]:
labels = hierarchy.fcluster(linkages, t=4, criterion='maxclust')
list(zip(*np.unique(labels, return_counts=True)))

In [0]:
check_clusters(seg_sub, labels)

In [0]:
linkages_unscaled = hierarchy.linkage(seg_sub, method='ward')
hierarchy.dendrogram(linkages_unscaled)
plt.show()

In [0]:
labels_unscaled = hierarchy.fcluster(linkages_unscaled, t=3,
                                     criterion='maxclust')
check_clusters(seg_sub, labels_unscaled)

In [0]:
def cluster_plot_raw(x, y, labels):
  for l in np.unique(labels):
    idx = labels == l
    plt.scatter(x[idx],
                y[idx],
                label=l)
  plt.legend()
  plt.xlabel(x.name)
  plt.ylabel(y.name)

In [0]:
cluster_plot_raw(seg_sub.age, seg_sub.income, labels_unscaled)

### 10.3.3 Mean-based clustering: K-*means*

In [0]:
def check_clusters(data, labels):
  print(list(zip(*np.unique(labels, return_counts=True))))
  
  return pd.pivot_table(data,
                        index=labels)

In [0]:
import numpy as np
from sklearn import cluster

np.random.seed(536)
centroids, labels, inertia = cluster.k_means(seg_sc, n_clusters=4)
check_clusters(seg_sub, labels)

In [0]:
centroids, k_labels_unscaled4, inertia = cluster.k_means(seg_sub,
                                                         n_clusters=4)
check_clusters(seg_sub, k_labels_unscaled4)

In [0]:
centroids, k_labels_unscaled3, inertia = cluster.k_means(seg_sub,
                                                         n_clusters=3)
check_clusters(seg_sub, k_labels_unscaled3)

In [0]:
import matplotlib.pyplot as plt
seg_sub.boxplot(column='income', by=k_labels_unscaled4)
plt.xlabel('Cluster')
plt.ylabel('Income')
plt.suptitle('') # Remove cluster id subtitle

In [0]:
from sklearn import decomposition
from matplotlib import cm

def cluster_plot(data_df, labels):
  p = decomposition.PCA(random_state=132, svd_solver='full')
  scaled_transformed = p.fit_transform(preprocessing.scale(data_df))
  for l in np.unique(labels):
    idx = np.where(labels == l)[0]
    plt.scatter(scaled_transformed[idx, 0],
                scaled_transformed[idx, 1],
                label=l)
  plt.legend()
  plt.title('First two components explain {}% of the variance'
            .format(round(100*p.explained_variance_ratio_[:2].sum())))
  plt.xlabel('First principle component')
  plt.ylabel('Second principle component')

cluster_plot(seg_sub, k_labels_unscaled4)

### 10.3.5 Model-based clustering: Gaussian Mixture Models

In [0]:
from sklearn import mixture

gmm4 = mixture.GaussianMixture(n_components=4,
                              covariance_type='full',
                              random_state=323).fit(seg_sub)
gmm4_labels = gmm4.predict(seg_sub)
gmm4.bic(seg_sub)

In [0]:
check_clusters(seg_sub, gmm4_labels)

In [0]:
gmm_n_test = [mixture.GaussianMixture(n_components=n,
                                      covariance_type='full',
                                      random_state=323)
                       .fit(seg_sub) for n in range(1,14)]
plt.plot(range(1, 14), [g.bic(seg_sub) for g in gmm_n_test])

In [0]:
gmm_n_v_test = {v: [mixture.GaussianMixture(n_components=n,
                                        covariance_type=v,
                                        random_state=323)
                       .fit(seg_sub) for n in range(1,14)]
                for v in ['full', 'tied', 'diag', 'spherical']}
gmm_n_v_test_bic = {v: [g.bic(seg_sub) for g in m]
                    for v, m in gmm_n_v_test.items()}
pd.DataFrame(gmm_n_v_test_bic).plot()

In [0]:
gmm_n_v_test = {v: [mixture.GaussianMixture(n_components=n,
                                        covariance_type=v,
                                        random_state=323)
                       .fit(seg_sc) for n in range(1,14)]
                for v in ['full', 'tied', 'diag', 'spherical']}
gmm_n_v_test_bic = {v: [g.bic(seg_sc) for g in m]
                    for v, m in gmm_n_v_test.items()}
pd.DataFrame(gmm_n_v_test_bic).plot()

In [0]:
gmm5 = mixture.GaussianMixture(n_components=5,
                               covariance_type='diag',
                               random_state=323).fit(seg_sub)
gmm5_labels = gmm5.predict(seg_sub)

In [0]:
check_clusters(seg_sub, gmm5_labels)

In [0]:
cluster_plot_raw(seg_sub.age, seg_sub.income, gmm5_labels)

In [0]:
n_obs = seg_sub.shape[0]
cluster_plot_raw(seg_sub.subscribe + .3*np.random.rand(n_obs),
                 seg_sub.is_female + .3*np.random.rand(n_obs),
                 gmm5_labels)